In [31]:
from lxml import etree as ET
import shutil
from subprocess import check_output
from pathlib import Path
import numpy as np
import os
#import matplotlib.pyplot as plt
import pandas as pd
#from shapely.geometry import Polygon,
import RClib as RClib



In [29]:
%load_ext autoreload
%autoreload 2
''
config = RClib.loadconfigs('.\config.json')
scandf = RClib.provide_scandf(inputdirectory=config['workspace'], imageformat = config['baseimageformat'])
previousscandf_path = Path(config['workspace']) / 'rcprocessingdf.pkl'
if previousscandf_path.is_file() and config['resume_processing']:
    previousscandf = pd.read_pickle(str(Path(config['workspace']) / 'rcprocessingdf.pkl'))
    previousscandf = previousscandf.set_index('id',drop=False)
    scandf = scandf.set_index('id',drop=False)
    print(previousscandf.columns)
    scandf  = pd.concat([scandf , previousscandf[previousscandf.columns.difference(scandf.columns)]], join='outer', axis=1)
    scandf.update(previousscandf)

print(scandf.columns)

scandf= scandf.apply(RClib.defineRawTherapeeOutput, foldername='02_TIF', axis=1)
scandf= scandf.apply(RClib.defineRealityCaptureOutput, foldername='03_RC', axis=1)
#rcsettings_template = pd.read_csv('.\\allRCsettings_template.csv', sep=';', header=0) 
#print(rcsettings_template)



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
E:\RCprocessing_test\PG_boat91_test\DSC00714.DNG
E:\RCprocessing_test\PG_boat91_test\DSC00715.DNG
E:\RCprocessing_test\PG_boat91_test\DSC00716.DNG
E:\RCprocessing_test\PG_boat91_test\DSC00717.DNG
E:\RCprocessing_test\PG_boat91_test\DSC00718.DNG
E:\RCprocessing_test\PG_boat91_test\DSC00719.DNG
E:\RCprocessing_test\PG_boat91_test\DSC00720.DNG
E:\RCprocessing_test\PG_boat91_test\DSC00721.DNG
E:\RCprocessing_test\PG_boat91_test\DSC00722.DNG
E:\RCprocessing_test\PG_boat91_test\DSC00723.DNG
E:\RCprocessing_test\PG_boat91_test\DSC00724.DNG
E:\RCprocessing_test\PG_boat91_test\DSC00725.DNG
E:\RCprocessing_test\PG_boat91_test\DSC00726.DNG
E:\RCprocessing_test\PG_boat91_test\DSC00727.DNG
E:\RCprocessing_test\PG_boat91_test\DSC00728.DNG
E:\RCprocessing_test\PG_boat91_test\DSC00729.DNG
E:\RCprocessing_test\PG_boat91_test\DSC00730.DNG
E:\RCprocessing_test\PG_boat91_test\DSC00731.DNG
E:\RCprocessing_test\PG_boat91

In [30]:

scandf2=pd.DataFrame()

for index,scan in scandf.iterrows():
    if not scan['imagedf'] is None:
        if 'pp3file' in scan.keys() and scan['pp3file']:    
            scan['imagedf'] = scan['imagedf'].apply(RClib.developwithRawTherapee, pp3filepath=scan['pp3file'][0],  outputfolderpath=scan['RToutputfolder'], axis=1) 
        scan = RClib.makeImagelist(scan, imagelistname=scan['id'])
        scan = RClib.createRCproject(scan)    
        if not 'commandlist_alignsettings' in scan['processingstate']:
            scan['commandlist_alignsettings']=Path(config['alignmentsettings'])
            RClib.executeRCCMDuseRCproject(scan, rccmdpathfield='commandlist_alignsettings' )
        if not 'align-rccmdpath' in scan['processingstate']:
            scan['commandlist_align'] = []
            scan['commandlist_align'].append('-add ' + str(scan['list_dev-img_path']))
            
            if config['use_gcps']:
                scan['commandlist_align'].append('-detectMarkers')
                if Path(config['constantgcps']).is_file() and Path(config['gcpsparamfile']).is_file():
                    scan['commandlist_align'].append('-importGroundControlPoints ' + str(config['constantgcps']) + ' ' + config['gcpsparamfile'])
                if not Path(config['constantgcps']).is_file() and Path(config['gcpsparamfile']).is_file() and scan['gcpsfile']:
                    scan['commandlist_align'].append('-importGroundControlPoints ' + str(scan['gcpsfile'][0]) + ' ' + config['gcpsparamfile'] )
            scan['commandlist_align'].append('-align')
            scan['commandlist_align'].append('-setReconstructionRegionAuto')
            scan['commandlist_align'].append('-exportReport ' + str(Path(scan['RCoutputfolder'] / Path(scan['id']+'.txt '))) + config['alignmentreport_template'])
            scan['commandlist_align'].append('-save')
            scan['commandlist_align'].append('-quit')
            scan = RClib.makeRCCMDfromListfield(scan, commandlistfield='commandlist_align', rccmdpathfield='align-rccmdpath')
            RClib.executeRCCMDuseRCproject(scan, rccmdpathfield='align-rccmdpath' )
        #scan['processingstatus'].append({'alignment_report': Path(scan['RCoutputfolder'] / Path(scan['id']+'.html '))})
        scandf2 = scandf2.append(scan)
#pd.write_csv(Path(config['workspace']) / 'rcprocessingdf.csv', sep=';')
scandf2.to_pickle(str(Path(config['workspace']) / 'rcprocessingdf.pkl'))

Inputfile:  E:\RCprocessing_test\PG_boat91_test\DSC00714.DNG
Expect file:  E:\RCprocessing_test\PG_boat91_test\02_PNG\DSC00714.tif
Inputfile:  E:\RCprocessing_test\PG_boat91_test\DSC00714.DNG
Expect file:  E:\RCprocessing_test\PG_boat91_test\02_PNG\DSC00714.tif
Inputfile:  E:\RCprocessing_test\PG_boat91_test\DSC00715.DNG
Expect file:  E:\RCprocessing_test\PG_boat91_test\02_PNG\DSC00715.tif
Inputfile:  E:\RCprocessing_test\PG_boat91_test\DSC00716.DNG
Expect file:  E:\RCprocessing_test\PG_boat91_test\02_PNG\DSC00716.tif
Inputfile:  E:\RCprocessing_test\PG_boat91_test\DSC00717.DNG
Expect file:  E:\RCprocessing_test\PG_boat91_test\02_PNG\DSC00717.tif
Inputfile:  E:\RCprocessing_test\PG_boat91_test\DSC00718.DNG
Expect file:  E:\RCprocessing_test\PG_boat91_test\02_PNG\DSC00718.tif
Inputfile:  E:\RCprocessing_test\PG_boat91_test\DSC00719.DNG
Expect file:  E:\RCprocessing_test\PG_boat91_test\02_PNG\DSC00719.tif
Inputfile:  E:\RCprocessing_test\PG_boat91_test\DSC00720.DNG
Expect file:  E:\RCpro

KeyboardInterrupt: 